In [ ]:
!mkdir data
!wget -P /content/data/ https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-train-detection-bbox.csv
!wget -P /content/data/ https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-train-detection-human-imagelabels.csv

mkdir: cannot create directory ‘data’: File exists
--2021-12-08 23:48:17--  https://storage.googleapis.com/openimages/challenge_2019/challenge-2019-train-detection-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.213.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 996981249 (951M) [application/octet-stream]
Saving to: ‘/content/data/challenge-2019-train-detection-bbox.csv’

challenge-2019-trai 100%[===================>] 950.79M   167MB/s    in 5.8s    

2021-12-08 23:48:23 (165 MB/s) - ‘/content/data/challenge-2019-train-detection-bbox.csv’ saved [996981249/996981249]



In [ ]:
!pip install aws-shell

In [ ]:
!aws s3 --no-sign-request cp s3://open-images-dataset/tar/train_7.tar.gz /content/data/images/

In [ ]:
!aws s3 --no-sign-request cp s3://open-images-dataset/tar/train_7.tar.gz /content/data/images/
!tar -xvzf /content/data/images/train_7.tar.gz -C /content/data/images/

FUNSD Dataset

In [ ]:
%%capture
!wget https://guillaumejaume.github.io/FUNSD/dataset.zip /content/
!unzip dataset.zip -d /content/

!cp -r /content/drive//My\ Drive/GoogleColab/Personal/Project/open_image_challenge/ open_image_challenge/

In [ ]:
%cd /content/open_image_challenge/

/content/open_image_challenge


In [ ]:
# Restart the kernel after running this cell
%%capture
!pip install -r requirements.txt

In [ ]:
import os.path
import copy

from sklearn.model_selection import train_test_split

from DataPrepararation import DataPreparation
from FUNSDDataset import FUNSDDataset
from RCNN import FasterRCNNModel
from LayoutLM import LayoutLMImageEmbeddingModel

import torch
from torch.utils.data import DataLoader
from torch.optim import SGD, AdamW

In [ ]:
data_folder = '/content/dataset/'

dataprep = DataPreparation(data_folder=data_folder)
df_train, df_test = dataprep.prepare_dataframe()

In [ ]:
df_train_m, df_train_p = train_test_split(df_train, train_size=0.66667)
df_train_m = df_train_m.reset_index(drop=True)
df_train_p = df_train_p.reset_index(drop=True)

In [ ]:
train_dataset_m = FUNSDDataset(
    image_data_folder=os.path.join(data_folder, 'training_data/images'),
    input_ids=df_train_m['input_ids'],
    token_type_ids=df_train_m['token_type_ids'],
    attention_mask=df_train_m['attention_mask'],
    imageid=df_train_m['imageid'],
    bbox=df_train_m['bboxes'],
    image_size=512,
    labels=df_train_m['labels']
)

train_dataset_p = FUNSDDataset(
    image_data_folder=os.path.join(data_folder, 'training_data/images'),
    input_ids=df_train_p['input_ids'],
    token_type_ids=df_train_p['token_type_ids'],
    attention_mask=df_train_p['attention_mask'],
    imageid=df_train_p['imageid'],
    bbox=df_train_p['bboxes'],
    image_size=512,
    labels=df_train_p['labels']
)

test_dataset = FUNSDDataset(
    image_data_folder=os.path.join(data_folder, 'testing_data/images'),
    input_ids=df_test['input_ids'],
    token_type_ids=df_test['token_type_ids'],
    attention_mask=df_test['attention_mask'],
    imageid=df_test['imageid'],
    bbox=df_test['bboxes'],
    image_size=224,
    labels=df_test['labels']
)


train_loader_m = DataLoader(train_dataset_m, batch_size=2)
train_loader_p = DataLoader(train_dataset_p, batch_size=1)
test_loader = DataLoader(test_dataset)

In [ ]:
rcnn_model = FasterRCNNModel(num_classes=5)
optim_rcnn = SGD(rcnn_model.parameters(), lr=5e-3, momentum=0.9, weight_decay=0.0005)

In [ ]:
device_ = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

rcnn_model.train()
rcnn_model.to(device_)
losses = []

epochs = 4
for epoch in range(epochs):
    print('-------------------------------------')
    print(f"Epoch:{epoch + 1}")
    losses.clear()
    for batch in train_loader_p:
        images = batch['images'].to(device_)
        orig_len = batch['orig_len']
        boxes = batch['bbox_rcnn']
        labels = batch['labels']

        targets = {
            'boxes': boxes,
            'labels': labels
        }

        images = list(image for image in images)
        targets = [{k: v[b][:orig_len[b]].to(device_) for k, v in targets.items()} for b in range(len(images))]

        optim_rcnn.zero_grad()
        output_losses = rcnn_model(images, targets)
        optim_rcnn.step()

        with torch.no_grad():
            loss = sum([output_losses[k].item() for k in output_losses.keys()])
            losses.append(copy.deepcopy(loss))

    print("Loss:", sum(losses)/len(losses))


-------------------------------------
Epoch:1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loss: 3.1010400993889196
-------------------------------------
Epoch:2
Loss: 3.0981275744503365
-------------------------------------
Epoch:3
Loss: 3.1004921312397347
-------------------------------------
Epoch:4
Loss: 3.0972614247864114


In [ ]:
del losses

In [ ]:
layoutlm_model = LayoutLMImageEmbeddingModel(num_classes=5, pretrained_rcnn_model=rcnn_model)
optim_layoutlm = AdamW(layoutlm_model.parameters(), lr=5e-5)

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing LayoutLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
layoutlm_model.to(device_)
layoutlm_model.train()

epochs = 4
losses = []
for epoch in range(epochs):
    print('-------------------------------------')
    print(f"Epoch:{epoch + 1}")
    losses.clear()

    for batch in train_loader_m:
        input_ids = batch['input_ids'].to(device_)
        attention_mask = batch['attention_mask'].to(device_)
        images = batch['images'].to(device_)
        bbox_layoutlm = batch['bbox_layoutlm'].to(device_)
        bbox_rcnn = batch['bbox_rcnn'].to(device_)
        labels = batch['labels'].to(device_)

        output = layoutlm_model(input_ids, bbox_layoutlm, bbox_rcnn, images, attention_mask, labels)

        optim_layoutlm.zero_grad()
        output.loss.backward()
        optim_layoutlm.step()

        with torch.no_grad():
            losses.append(copy.deepcopy(output.loss.item()))

    print("Loss:", sum(losses)/len(losses))


-------------------------------------
Epoch:1
Loss: 0.5779041320085525
-------------------------------------
Epoch:2
Loss: 0.30978871643543243
-------------------------------------
Epoch:3
Loss: 0.2594358536601067
-------------------------------------
Epoch:4
Loss: 0.19255270302295685


In [ ]:
layoutlm_model.eval()

y_preds = []
y_true = []
for batch in test_loader:
    input_ids = batch['input_ids'].to(device_)
    attention_mask = batch['attention_mask'].to(device_)
    images = batch['images'].to(device_)
    bbox_layoutlm = batch['bbox_layoutlm'].to(device_)
    bbox_rcnn = batch['bbox_rcnn'].to(device_)
    labels = batch['labels']

    output = layoutlm_model(input_ids, bbox_layoutlm, bbox_rcnn, images, attention_mask)
    
    pred = torch.argmax(output.logits, dim=-1)
    y_preds.append(pred.cpu().tolist())
    y_true.append(labels.cpu().tolist())


In [ ]:
accuracy = torch.sum(y_true_tensor == y_pred_tensor) / torch.numel(y_true_tensor)
print(accuracy)

tensor(0.9436)
